In [ ]:
################################################################################################
# <문장 임베딩 예시>
# sentence-bert와 Faiss 라이브러리를 이용하여 검색 MRR(Mean Reciprocal Rank)측정 예시 3
# - contexts(문서)들를 sub 문장으로 나누고, 나눈 sub 문장들에 대해 임베딩벡터를 구해서 Faiss 인덱싱 생성(contexts(문서)별 1개 Faiss 인덱싱 생성)
# - 쿼리문장 임베딩벡터를 구해서 각 contexts(문서) Faiss 인덱스에서 sub 문장 벡터와 비교하여 유사도 스코어 계산.
# - 말뭉치는  KorQuAD_v1.0 파일과 aihub(aihub.or.kr) 뉴스 기사 기계독해 데이터 QuA 파일 3종류 이용

# 1. 검색모델 로딩
# 2. JSON 파일 로딩 후 df 만듬
# 3. 문장(문서)들을 sub 문장으로 분리 
# 4. 문단(문서)별 문장들의 토큰 평균들에 대해 Faiss 임베딩 생성
# 5. 쿼리 후 가장 유사한 문장(문서)의 sub 문장 평균 구함
# 6. MRR 계산
#
#-------------------------------------------------------------------------------
# 쿼리가:단어일때 검색율
# -문장: 33%(쿼리문장:86.8%), 문장분리=47%, 문장+단어(50개)=53.60%/(70개)=59%, 128차원(50개)=50.40%/(100개)=55.8% =>결국 70개 768차원 벡터 생성이 가장 좋음(단어query: 59%, 문장query: 80%)
# -문장 토큰 고정 분할 : 5개분할=54.6%(쿼리문장:90%)/10개=46.6%(91%)/15개=41.2%/20개==40.6%(90.4%)/30개=38.60%
# -문장 토큰 가변 분할 : (7,8,10)=50.8%(90.2%)
# -BM25=Mecab 적용시: 85%, 적용안할때: 50%
#-------------------------------------------------------------------------------
# sklenarn 으로 cosine 확인 예제
# from sklearn.metrics.pairwise import cosine_similarity
# cosine_sim = cosine_similarity([embed_querys[0]], [embed_querys_1[0]]) # (1,768) 식에 2차원 배열입력되어야 함.
#-------------------------------------------------------------------------------
################################################################################################

import faiss
import numpy as np
import pandas as pd
import time

from os import sys
sys.path.append('../../')
from myutils import GPU_info, seed_everything, mlogging, bi_encoder

logger = mlogging(loggername="MRR-VOCAB", logfilename="../../../log/MRR-VOCAB")
device = GPU_info()
#device = 'cpu'  # cpu 테스트 할때

#------------------------------------------------------------------------------------
# 0. param 설정
#------------------------------------------------------------------------------------
seed = 111
query_num = 500             # 쿼리 최대 갯수: KorQuAD_v1.0_dev.json 최대값은 5533개임, 0이면 모든 5533개 쿼리함.
search_k = 5                # FAISS 검색시, 검색 계수(5=쿼리와 가장 근접한 5개 결과값을 반환함)
avg_num = 1                 # 쿼리에 대해 sub 문장들중 최대 scorce를 갖는 문장을 몇개 찾고 평균낼지.(3=쿼리에 가장 유사한 sub문장 3개를 찾고 평균을 냄)
faiss_index_method = 0      # 0= Cosine Similarity 적용(IndexFlatIP 사용), 1= Euclidean Distance 적용(IndexFlatL2 사용)

# 토큰 임베딩 관련 param
IS_EMBED_DEVIDE = True      # True=문단의 여러 문장을, 토큰 단위로 분리후 벡터 구해서 인덱스 만듬/False=문단의 여러문장을 하나의 벡터를 구해서 인덱스 만듬.
EMBED_DIVIDE_LEN = [7,8,10] #5 # 문장을 몇개(토큰)으로 분리할지 (7,8,10) 일때 성능 좋음=>50.8%
MAX_TOKEN_LEN = 50          # 최대 몇개 token까지만 임베딩 할지
#------------------------------------------------------------------------------------
# param 인자 범위 체크
if faiss_index_method > 1 or faiss_index_method < 0:
    raise ValueError(f"faiss_index_method = {faiss_index_method} is not bad!!")
    
seed_everything(111)

In [ ]:
#-------------------------------------------------------------------------------------
# 1. 검색모델 로딩
# => bi_encoder 모델 로딩, polling_mode 설정
#-------------------------------------------------------------------------------------
from myutils import bi_encoder

bi_encoder_path = "bongsoo/kpf-sbert-v1.1"#"bongsoo/kpf-sbert-v1.1" # kpf-sbert-v1.1 # klue-sbert-v1 # albert-small-kor-sbert-v1.1
pooling_mode = 'mean' # bert면=mean, albert면 = cls
out_dimension = 0    # 출력 임베딩 크기 지정 : 0=기본 모델 임베딩크기(768), 예:128=128 츨력임베딩 크기 

word_embedding_model, bi_encoder = bi_encoder(model_path=bi_encoder_path, max_seq_len=512, do_lower_case=True, 
                                              pooling_mode=pooling_mode, out_dimension=out_dimension, device=device)
#bi_encoder.to(device)

In [ ]:
#--------------------------------------------------
# 테스트 같은 단어도 문장이 다르면 임베딩이 달라진다??
#--------------------------------------------------
'''
from myutils import embed_text
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, util

text = "배를 맛있게 먹다"
tmp = embed_text(model=bi_encoder, paragraphs=[text], token_embeddings=True)
print(tmp[0].size())
tmp1=tmp[0].cpu().numpy()
print(tmp1.shape)

tokenizer = word_embedding_model.tokenizer
token = tokenizer.tokenize(text)
print(token)

#tmp1 = [tmp[0].cpu().numpy().mean(axis=0)]
#embed1 = np.array(tmp1).astype('float32')
#embed1 = embed_text(model=bi_encoder, paragraphs=[text], token_embeddings=False)
#print(type(embed1))
#print(embed1.shape)

text = "배가 너무 달콤하다"
tmp = embed_text(model=bi_encoder, paragraphs=[text], token_embeddings=True)
print(tmp[0].size())
tmp2=tmp[0].cpu().numpy()
print(tmp2.shape)

tokenizer = word_embedding_model.tokenizer
token = tokenizer.tokenize(text)
print(token)

text = "배를 타고 여행을 간다"
tmp3 = embed_text(model=bi_encoder, paragraphs=[text], token_embeddings=True)
tt = [tmp3[0].cpu().numpy().mean(axis=0)]
tmp3 = np.array(tt).astype('float32')
print(tmp3.shape)


score = cosine_similarity([tmp1[1]], tmp3) # (1,768) 식에 2차원 배열입력되어야 함.
#score = util.semantic_search([tmp2[1]], tmp3, score_function=util.dot_score)

print(f'*유사도:{score}')
'''

In [ ]:
#-------------------------------------------------------------------------------------------------------
# 2. JSON 파일 로딩 후 df 만듬
# => KorQuAD_v1.0_dev.json 파일 로딩 하여, 각 항목별 리스트를 출력한후, contexts df, questions df를 만듬.
#-------------------------------------------------------------------------------------------------------
from myutils import read_korquad_v1_json, read_aihub_qua_json

# list 데이터들을 dataframe으로 만듬
'''
contexts = [
        [1, '독도 독도 독도 해역 헬기 추락사고가 발생한 지 열하루가 지났지만 실종자 추가 발견 소식은 들려오지 않고 있다. 헬기 동체 잔해물과 부유물 등은 발견되고 있지만 정작 실종자들은 발견하지 못해 수색이 장기화될 것이라는 우려가 현실이 될 조짐을 보여 실종자 가족들의 애를 태우고 있다.범정부현장수습지원단(지원단)은 10일 오전 10시 브리핑에서 이날 오전까지 독도해역 수색 결과 4점의 부유물을 추가 발견, 인양했다고 밝혔다'], 
        [11,'대구시는 11일 내년 정부 예산 3조1330억원을 확보해 전년보다 2%(611억원) 늘어났다고 밝혔다.그러나 올해 국비 증액 규모(1817억원)와 비교해 절반 수준에 그치며, 물산업클러스터 r&d 사업(200억), 국립청소년진로직업체험 수련원 건립 등은 한푼도 반영되지 않았다.'],
        [21,'자유한국당 (이름) 의원이 "공수처는 정권 유지를 위한 수단으로, 공수처가 있었으면 조국 수사를 못했을 것"이라고 주장했다.최 의원은 18일 대구 호텔수성에서 열린 대구 경북중견언론인모임 <아시아포럼21> 토론회에서 "검찰은 정권 말기가 되면 정권에 칼을 들이댔다. 공수처는 검찰의 칼끝을 회피하기 위한 것"이라며 이같이 말했다'],
        [2,'27일 오전 천연기념물(201-2호)이며 멸종위기 2급인 큰고니 떼가 경북 포항시 북구 흥해읍 샛강에 날아들었다.이날 관측된 큰고니는 어미 8마리와 새끼 3마리다.앞서 지난 19일 큰고니 8마리가 올들어 처음으로 관측됐다.큰고니들은 포항시 흥해읍 샛강에서 수초 등을 먹고 휴식한 후 대구 안심습지와 낙동강 하구 등지로 날아갈 것으로 보인다'],
        [3,'3일 대구와 경북지역은 구름이 많겠으며, 동해안과 북부 내륙에는 비가 내릴 것으로 예상된다.대구기상청에 따르면 중국 북동지방에서 남하하는 고기압의 가장자리에 들어 구름이 많겠으며, 경북 북부 내륙은 북쪽을 지나는 약한 기압골의 영향을 받아 오후부터 비가 약하게 내리거나 빗방울이 떨어지겠다.'],
        [31,'각급 정부기관의 통신망을 관리할 국가정보자원관리원 대구센터가 31일 대구 동구 도학동에서 착공했다.4312억원을 투입해 2021년 8월 준공 예정인 대구센터는 86개 기관의 서비와 장비를 운용하게 된다.정부통합전산센터에서 명칭이 변경된 국가정보자원관리원은 현재 대전본원과 광주센터를 운영 중이다.대구센터는 급변하는 행정환경과 4차 산업혁명 시대에 맞춰 클라우드, 빅데이터 등 신기술이 접목된 지능형 전산센터로 구축된다'],
        [4,'26일 오후 11시44분쯤 대구 달서구 신당동의 한 아파트에서 원인 모를 불이 나 주민 1명이 다치고 수십명이 대피했다.27일 소방당국에 따르면 신고를 받고 소방차 24대와 소방대원 74명이 출동해 20분만에 불길을 잡았다.이 불로 아파트 2층에 사는 a씨(53)가 온 몸에 화상을 입어 병원으로 이송됐으며, 연기에 놀란 주민 50여명이 대피하는 소동을 빚었다.']
       ]
df_contexts = pd.DataFrame(contexts, columns=['contextid','context'])
print(df_contexts['context'].values)
'''

# KorQuAD_v1.0 혹은 aihub 뉴스 기사 기계독해 데이터 QuA 파일을 불러옴.
jsonfile = './data/VL_unanswerable.json' # VL_unanswerable.json # VL_text_entailment.json # VL_span_inference.json # KorQuAD_v1.0_dev.json
contexts, questions, answers, contextids, qcontextids = read_aihub_qua_json(jsonfile) # read_aihub_qua_json(jsonfile)

# list들을 zip 으로 묶고, dataframe 생성함
# context, contextid를 묶어서 context df 만듬.
df_contexts = pd.DataFrame((zip(contexts, contextids)), columns = ['context','contextid'])

# question, answer, contextids를 묶어서 question df 만듬
df_questions = pd.DataFrame((zip(questions, answers, qcontextids)), columns = ['question','answer', 'contextid'])

In [ ]:
#-------------------------------------------------------------------------------------------------------
# 3. 문장(문서)들을 sub 문장으로 분리
# =>df_contexts에 대해 각 문장별 '.'(마침표) 로 구분해서 sub 문장을 만듬. 혹은 kss 이용해서 sub 문장을 만듬
#-------------------------------------------------------------------------------------------------------
from tqdm.notebook import tqdm
import kss

sub_contexts = []

for context in tqdm(df_contexts['context'].values.tolist()):
    #sentences = [sentence for sentence in context.split('.') if sentence != '' and len(sentence) > 10]  # '.'(마침표) 로 구분해서 sub 문장을 만듬.
    sentences = [sentence for sentence in kss.split_sentences(context) if sentence != '' and len(sentence) > 10] # kss 이용해서 sub 문장을 만듬
    # 만약 sentences(sub 문장) 가 하나도 없으면 원본문장을 담음
    if len(sentences) <= 1:
        sentences = [context]
    #elif len(sentences) > 10:
    #    sub_contexts.append(sentences[0:9])  # 10줄 이상이면 10줄만 입력
    else:                    
        sub_contexts.append(sentences) 
    
print(sub_contexts[1])

In [ ]:
#-------------------------------------------------------------------------------------
# 4. 문단(문서)별 문장들의 토큰 평균들에 대해 Faiss 임베딩 생성
# - 각 문단의 문장들에 대해 토큰들의 평균값을 구하고, Faiss에 임베딩 추가함.
#-------------------------------------------------------------------------------------
from myutils import embed_text, fassi_index, divide_arr_avg, divide_arr_avg_exten
 
EMBED_CONTEXTS = sub_contexts#sub_vocab_contexts # 임베딩을 구해 인덱스에 추가할 문장들 1차원 리스트 (예:['오늘은 좋다','날씨가 흐리다','제주도 날씨',...])
MAX_EMBED_LEN = 70                  # 한문장단 최대 인덱스할 문장+단어 계수

faissindexlist = []

print(f'*faiss 인덱싱 시작 => 방식: 토큰 분리 임베딩:{IS_EMBED_DEVIDE}(분리수:{EMBED_DIVIDE_LEN}개 토큰)/{faiss_index_method}(0=코사인, 1=유클리드)')
    
start = time.time()
embed_context = []
embed_len = []
for idx, embed_context in enumerate(tqdm(EMBED_CONTEXTS)):
    if len(embed_context) > MAX_EMBED_LEN:
        embed_context = embed_context[0:MAX_EMBED_LEN]
        #print(f'{idx}\n{embedding_context}')

    # 한 문단에 대한 50개 문장 배열들을 한꺼번에 임베딩 처리함
    if IS_EMBED_DEVIDE == False:
        embeddings = embed_text(model=bi_encoder, paragraphs=embed_context, return_tensor=False)
    else:
        #------------------------------------------------------------------------------------------------------------------------
        # 한 문단에 대한 문장들의 토큰을 ?개씩 나누고 비교.
        # - 한 문단에 대한 문장들에 대해 [tensor(250,768), tensor(243,768), tensor(111,768),..] tensor 리스트 타입으로 벡터 생성됨.
        token_embeds = embed_text(model=bi_encoder, paragraphs=embed_context, token_embeddings=True, return_tensor=False)
        token_embed_arr_list = []
        tcount = 0
        # tensor(250,768) 한문장 토큰 임베딩 얻어와서, 각 ?개 토큰씩 평균을 구함.
        for token_embed in token_embeds:
            if tcount >= MAX_TOKEN_LEN: 
                break
            token_embed_arrs = token_embed.cpu().numpy().astype('float32')
            
            # token_embed_divide_arrs = divide_arr_avg(embed_arr=token_embed_arrs, divide_len=EMBED_DIVIDE_LEN)# EMBED_DIVIDE_LEN 만큼 자르면서 문장 토큰 평균을 구함
            token_embed_divide_arrs = divide_arr_avg_exten(embed_arr=token_embed_arrs, divide_arrs=EMBED_DIVIDE_LEN) # 5,7,10 씩 자르면서 문장 토큰 평균을 구함

            # 평균 구한 토큰들을 token_embed_arr_list 리스트에 담아둠.(50보다 크면 50개만 담음)           
            for idx, token_embed_divide_arr in enumerate(token_embed_divide_arrs):
                token_embed_arr_list.append(token_embed_divide_arr)
                tcount +=1
                if tcount >=MAX_TOKEN_LEN:
                    break
                
        embeddings = np.array(token_embed_arr_list)
        #------------------------------------------------------------------------------------------------------------------------
        
    embed_len.append(len(embeddings))
    
    # Faiss index 생성하고 추가 
    index = fassi_index(embeddings=embeddings, method=faiss_index_method)
    faissindexlist.append(index)
    
print(f'*임베딩 시간 : {time.time()-start:.4f}')
print(f'*임베딩 shape : {embeddings.shape}')

ecount = 0
for elen in embed_len:
    if elen >= 50:
        ecount+=1
        
print(f'*인덱스별 임베딩 수 : {embed_len}/*최대값:{max(embed_len)}/*50 <= 계수:{ecount}')

In [ ]:
#-------------------------------------------------------------------------------------
# 5. 쿼리 후 가장 유사한 문장(문서)의 sub 문장 평균 구함
# => 쿼리 문장에 임베딩값을 구하고, 이후 Faiss sub 문장 임베딩과 비교하여 가장 유사한 문장 avg_num개 를 찾고 평균을 구함
#    이후 평균값이 가장 큰 search_k 갯수 만 쿼리에 대한 예측 결과 리스트로 만듬
#-------------------------------------------------------------------------------------
from myutils import df_sampling, sum_of_array_2d, index_of_list,split_sentence_list

# 쿼리문장 => mecab으로 단어단위로 쪼개고, 이후 단어간 임베딩 시작 
#user_querys = ["독도에서 사고가 나서 실종자가 발생했다.", "오늘 날씨가 흐리고 비가 오겠다."]

# 쿼리 샘플링함.
if query_num == 0:   # query_num = 0 이면 모든 쿼리(5533개)
    user_querys = df_questions['question'].values.tolist()
else:   # query_num > 0이면 해당 계수만큼 랜덤하게 샘플링하여 쿼리 목록을 만듬.
    df_questions = df_sampling(df=df_questions, num=query_num, seed=seed)
    user_querys = df_questions['answer'].values.tolist()
  
print(f'Query-----------------------------------------------------')
print(user_querys[0:3])


bi_predictions_list = []

# 문장으로 비교할때
if IS_EMBED_DEVIDE == False:
    embed_querys = bi_encoder.encode(user_querys)
else:
    #------------------------------------------------------------------
    # 한 문단에 대한 문장들의 평균을 구함.
    token_query_embeds = embed_text(model=bi_encoder, paragraphs=user_querys, token_embeddings=True, return_tensor=False)
    token_query_embed_arr_list = []
    # 쿼리 문장들의 토큰들의 평균을 구함.
    for token_query_embed in token_query_embeds:
        tmp = token_query_embed.cpu().numpy().astype('float32')
        token_query_embed_arr_list.append(tmp.mean(axis=0))
        
    embed_querys = np.array(token_query_embed_arr_list)  # 리스트를 배열로 변환  
#------------------------------------------------------------------


if faiss_index_method == 0:
    faiss.normalize_L2(embed_querys)          # *cosine유사도 구할때는 반드시 normalize 처리함.
 
for embed_query in tqdm(embed_querys):
    embed_query = [embed_query]
    
    max_values_list = []
    for count, index in enumerate(faissindexlist):
        distance, idx = index.search(np.array(embed_query).astype("float32"), k=avg_num) # avg_num 계수 만큼 유사한 sub문장을 찾음
        avg_distance = distance.mean(axis=1) # 검색된 sub 문장들에 대해 평균을 구함
        max_values_list.append(avg_distance[0])
        
    # 문장별 스코어 최대값 리스트에서 최대값을 갖는 항목 search_k개 index만 출력 함.
    # faiss_index_method=0 => cosine유사도 구할때는 max 값을 갖는 index 출력함.(예: search_k=3일때, np.array([21,11,41,51,31]) 일때 출력 [3,2,4])
    # faiss_index_method=1 => 유클리드 거리 로 할때는 min 값을 갖는 index 출력함.(예: search_k=3일때, np.array([21,11,41,51,31]) 일때 출력 [1,0,4])
    indices = index_of_list(listdata=max_values_list, k=search_k, bmin=faiss_index_method)  
    
    # 예측검색결과 contextid값들을 리스트로 만듬.
    tmp_bi_predictions_list = []
    for indice in indices:
        tmp_bi_predictions_list.append(df_contexts["contextid"][indice])
        #print(f'*인덱스:{indice}/contextid:{df_contexts["contextid"][indice]}-----------------------------------------------------------')
        #print('*총합/평균 : {:.4f}/{:.4f}'.format(max_values_list[indice], float(max_values_list[indice]/len(mecab_query))))
        #print(df_contexts['context'][indice])
        #print(mecab_contexts[indice])
    
    # 2D 예측검색결과 리스트에 추가 
    bi_predictions_list.append(tmp_bi_predictions_list)

print(bi_predictions_list[0:3])


In [ ]:
#--------------------------------------------------------------------------------------------------
# 6. MRR 계산
# => 정답 리스트[2,3,1,4] 과 예측검색리스트[[1,2,5,1],[3,4,2,1],[6,5,4,1], [2,3,4,1]]를 입력하여 MRR 스코어 구함
##--------------------------------------------------------------------------------------------------
from myutils import mean_reciprocal_rank

# 정답, 여기서는 contextid를 리스트로 만듬.
ground_truths_list = df_questions['contextid'].values.tolist()
#print(f'gtlen:{len(ground_truths_list)}')
#print(ground_truths_list[0:9])

logger.info(f'--------------------------------------------------------------------------')
logger.info('json_file:{}'.format(jsonfile))
logger.info(f'faiss 인덱싱 방식: 토큰 분리 임베딩:{IS_EMBED_DEVIDE}(분리수:{EMBED_DIVIDE_LEN}개 토큰)/{faiss_index_method}(0=코사인 유사도, 1=유클리드 거리)')
logger.info('*avg_num:{}/search_k:{}/query_num:{}/out_dimension:{}/MAX_EMBED_LEN:{}'.format(avg_num, search_k, query_num, out_dimension, MAX_EMBED_LEN))

# MRR 계산
bi_ranks, bi_score = mean_reciprocal_rank(ground_truths_list, bi_predictions_list)

# BI-MRR 출력
logger.info(f'----------------------------------------------------------------------------')
logger.info('*BI-ENCODER:{}'.format(bi_encoder_path))
logger.info('*BI-MRR:{:.4f}'.format(bi_score))
logger.info(f'*Ranks({len(bi_ranks)}):{bi_ranks[0:20]}')
    
# 검색 한 계슈
#logger.info(f'---------------------------------------------------------------------------')
search_count = 0
for item in bi_ranks:
    if item != 0:
        search_count += 1
    
logger.info('*검색률: {}/{}({:.2f}%)'.format(search_count, len(bi_ranks), (search_count/len(bi_ranks))*100))
logger.info(f'---------------------------------------------------------------------------')